In [1]:
import pandas as pd
import numpy as np
import json
import os
from collections import defaultdict, deque
import pickle
from IPython import get_ipython
from tqdm.auto import tqdm
import gc
import subprocess
from IPython.display import clear_output
import matplotlib.pyplot as plt
import wandb

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cuda


# Train

In [11]:
%run train.py --lr 0.0025 --layers 1024 --num_timesteps 1000 --is_y_cond --save_as 'dqn_test' 

Index(['Month_sin', 'Month_cos', 'Hour_sin', 'Hour_cos', 'Day_sin', 'Day_cos',
       'Coord X', 'Coord Y', 'Duration', 'Incident', 'Month', 'Day', 'Hour'],
      dtype='object')
Index(['Month_sin', 'Month_cos', 'Hour_sin', 'Hour_cos', 'Day_sin', 'Day_cos',
       'Coord X', 'Coord Y', 'Duration', 'Incident'],
      dtype='object')
K [0]
features 9
{'d_in': np.int64(9), 'is_y_cond': True, 'num_classes': 56, 'rtdl_params': {'d_layers': [1024, 1024], 'dropout': 0.0}, 'dim_t': 128}
mlp
label embedding Embedding(56, 128)
diffusion ready
Model trained


# Sample

In [3]:
for i in range(1): # génère i datasets
    print("iteration", i)
    filename = f"df_fake_{i}_test.pkl"
    
    process = subprocess.Popen(
        ["python", "sample.py", 
         "--load_as", 'dqn_test', 
         "--save_sample_as", filename, 
         "--os_factor", "3", 
         "--pressure", "1.2",
         "--to_keep", "40", 
         "--value_span", "100"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1  # pour forcer le flush ligne par ligne
    )

    # process.wait() # évite le lazy

    line_count = 0
    stdout_lines = deque(maxlen=5000)
    for line in process.stdout:
        stdout_lines.append(line)
        line_count += 1
        if line_count % 100 == 0:
            clear_output(wait=True)
    
        print(line.strip())

    print(f"--- {filename} done---\n")

iteration 0
os factor: 3 sample batch size: 8192 pressure: 1.2 variability: 0.02
[0]
features 9
{'d_in': np.int64(9), 'is_y_cond': True, 'num_classes': 56, 'rtdl_params': {'d_layers': [1024, 1024], 'dropout': 0.0}, 'dim_t': 128}
mlp
label embedding Embedding(56, 128)
params loaded
diffusion ready
Index(['area_name', 'count', 'new_samples', 'perc.', 'delta'], dtype='object')
new samples OK
(63696, 14)
Index(['Coord X', 'Coord Y', 'Duration', 'Day', 'Month', 'Hour', 'Incident',
'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos',
'area_name'],
dtype='object')
dataset sampled
--- df_fake_0_test.pkl done---



# Generate environment

In [ ]:
# lister les fichiers samplés à concaténer

# df_fake_0.pkl df_fake_1.pkl df_fake_2.pkl df_fake_3.pkl df_fake_4.pkl df_fake_5.pkl df_fake_6.pkl df_fake_7.pkl df_fake_8.pkl df_fake_9.pkl

In [23]:
os.chdir("./Scripts/")

In [4]:
%%time

%run generate_environment.py --sample_list df_fake_0_test.pkl --save_as "df_pc_fake_1y_p12_test.pkl"

is probabilistic departure False


/media/michael/Fichiers/Fac/Thèse 1/Code/Scripts/generate_environment.py:443: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_materiel = pd.read_csv("materiel_2018.csv", sep=";")
/media/michael/Fichiers/Fac/Thèse 1/Code/Scripts/generate_environment.py:449: DtypeWarning: Columns (22,23,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lieu = pd.read_csv("dbo.LIEU.csv", sep=";")


window real: 53088
start_year 2018 start_inter 1 end_inter 53088
362 197
Day
1      219
2      133
3      154
4      165
5      126
      ... 
361    106
362     86
363    130
364    131
365    138
Name: count, Length: 365, dtype: int64
real 106176 done
window fake: 63696
start_year 2018 start_inter 1 end_inter 63696
(63696, 18)
(63696, 19)
1 177
Day
1      124
2      164
3      178
4      162
5      166
      ... 
361    175
362    170
363    173
364    157
365    124
Name: count, Length: 365, dtype: int64
(63696, 21)
(127392, 18)
df_fake_0_test.pkl 127392 done
Index(['num_inter', 'date', 'PDD', 'departure', 'zone', 'Duration', 'Month',
       'Day', 'Hour', 'Minute', 'Coord X', 'Coord Y', 'Month_sin', 'Month_cos',
       'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos'],
      dtype='object')
global fake done 127392
Planning done
CPU times: user 3min 54s, sys: 901 ms, total: 3min 55s
Wall time: 3min 56s


# Metrics

In [2]:
os.chdir('./Reward_weights')

metrics = ['v_degraded', 
           'v1_not_sent_from_s1', 
           'v3_not_sent_from_s3', 
           'v_not_found_in_last_station', 
           'z1_VSAV_sent', 
           'rupture_ff',
          'VSAV_disp']

for m in metrics:

    dic_tarif_sent_disp = {'v_required': 0,
                    'v_sent': 0,
                    'v_sent_full':0,
                    'v_degraded':0,
                    'cancelled':0, #cancel departure
                    'function_not_found':0,
                    'v1_not_sent_from_s1':0,
                    'v3_not_sent_from_s3':0,
                    'v_not_found_in_last_station':0,
                    'ff_required':0,
                    'ff_sent':0,
                    'rupture_ff':0,       
                    'z1_VSAV_sent': 0,
                    'z1_FPT_sent': 0,
                    'z1_EPA_sent': 0,
                     'VSAV_needed':0,
                     'FPT_needed':0,
                     'EPA_needed':0,
                     'VSAV_disp':0,
                     'FPT_disp':0,
                     'EPA_disp':0,
                    'skill_lvl':0
                    } 

    dic_tarif_sent_disp[m] = -100

    if m == 'v_degraded':
        
        dic_tarif_sent_disp['v_sent_full'] = 10


    with open(f"rw_"+ m +".json", "w") as f:
        json.dump(dic_tarif_sent_disp, f)

os.chdir("../")

# Simulation

In [ ]:
cmd = [
    "python3", "-u", "simulation_start.py",
    "--dataset", "df_pc_real.pkl",
    "--start", "1",
    "--end", "53088",
    "--constraint_factor_veh", "3",
    "--constraint_factor_ff", "1",
    "--is_best",
    "--save_metrics_as", f"sim_sent_disp_bst"
]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

# Agent params

In [ ]:
action_size = 80
train_seed = 41

os.chdir("./Data")


hyper_params = {"state_size" : (action_size + 2) *40,
                "action_size" : action_size,
                "layer_type" : "ff", # noisy else ff
                "layer_size" : 1024,
                "num_layers" : 8,
                "use_batchnorm" : True,
                "n_steps" : 20, #5, 1, 50, 64
                "batch_size" : 256, #512, 256, 64, 32, 16, 8
                "buffer_size" : 100000, #100000, 128, 5000, 10000, 65 buffer_size > batch_size
                "update_every" : 32, #1, 200, 500, 64, 32 Q updates
                "per" : 1, # 0 for curiosity > 0, else 1, 2
                "rdm" : 0, # only if not per
                "munchausen" : 1, #1
                "curiosity" : 0, #Adds intrinsic curiosity to the extrinsic reward. 0 - only reward/ no curiosity, 
                                                                                    #1 - reward and curiosity, 
                                                                                    #2 - only curiosity
                "curiosity_size" : 1024,
                "lr" : 1e-4, #1e-3, 5e-4, 5e-3, 1e-4
                "lr_dec" : 1, #0, 1, 2, 3
                "entropy_tau" : 0.03, #0.03, 0.05 idem #-  Munch param
                "entropy_tau_coeff" : 1e-2, #1e-2 #-  Munch param
                "lo" : -1, #-  Munch param
                "alpha" : 0.9, #-  Munch param
                "gamma" : 0.99,
                "tau" : 0.005, #1e-2, 5e-3
                "N" : 32,# Number of quantiles 32, 64
                "entropy_coeff" : 0.001,
                "decay_update" : 100,
                "device" : str(device),
                "seed" : train_seed}


json.dump(hyper_params, open("hyper_params.json", "w"))

os.chdir("../")

# Agent model

In [ ]:
model = "fqf"
years = "10y"

cmd = [
    "python3", "-u", "agent_run.py",
    "--model_name", f"agent_{model}_{years}",
    "--agent_model", model,
    "--hyper_params", "hyper_params.json",
    "--reward_weights", f"rw_sent_disp.json",
    "--dataset", f"df_pc_fake_{years}.pkl",
    "--start", "1",
    "--end", "530880",
    "--eps_start", "1",
    "--constraint_factor_veh", "3",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"agent_metrics_{model}_{years}",
    "--train",

]
# "--load"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

# Stats

In [ ]:
folder_path = "./Plots"

data = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith(".pkl"):
        full_path = os.path.join(folder_path, file_name)
        with open(full_path, "rb") as f:
            d = pickle.load(f)
            name = os.path.splitext(file_name)[0]
            data[name] = d

df = pd.DataFrame.from_dict(data, orient='index')

# Plots

In [ ]:
%%time

%run plot_evo.py \
agent_metrics_v_degraded_r100_cf3_reward_real.npy \
sim_metrics_r100_cf3_reward_real.npy \
--interpolation 1000